In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy import stats
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.multitest import multipletests


%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
data_path = "data/charting-m-points.csv"
df = pd.read_csv(data_path, encoding="ISO-8859-1")

/Users/jamesmoss/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,8,9,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<IPython.core.display.Javascript object>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439988 entries, 0 to 439987
Data columns (total 43 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   match_id       439988 non-null  object 
 1   Pt             439988 non-null  int64  
 2   Set1           439988 non-null  int64  
 3   Set2           439988 non-null  int64  
 4   Gm1            439987 non-null  float64
 5   Gm2            439987 non-null  float64
 6   Pts            439988 non-null  object 
 7   Gm#            439987 non-null  object 
 8   TbSet          439988 non-null  object 
 9   TB?            439988 non-null  object 
 10  TBpt           15632 non-null   float64
 11  Svr            439988 non-null  int64  
 12  Ret            439988 non-null  int64  
 13  Serving        439375 non-null  object 
 14  1st            439988 non-null  object 
 15  2nd            166825 non-null  object 
 16  Notes          26591 non-null   object 
 17  1stSV          439988 non-nul

<IPython.core.display.Javascript object>

In [32]:
pd.set_option("display.max_columns", None)
# df["match_id"].head(5)
# df.head(5)

<IPython.core.display.Javascript object>

In [29]:
df["year"] = df["match_id"].str[0:4].astype(int)
df['player1_id']=None
df['player2_id']=None
df['player2_id']=df["match_id"].str.split("-").str[-1]
df['player1_id']=df["match_id"].str.split("-").str[-2]


<IPython.core.display.Javascript object>

In [31]:
df_trim = df[
    [
        "match_id",
        "player1_id",
        "player2_id",
        "year",
        "Svr",
        "isDouble",
        "isAce",
        "PtWinner",
        "isSvrWinner",
    ]
]
df_trim.head()

,match_id,player1_id,player2_id,year,Svr,isDouble,isAce,PtWinner,isSvrWinner
0,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,1,1
1,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,2,0
2,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,2,0
3,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,1,1
4,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,1,1


<IPython.core.display.Javascript object>

### Make variables that indicate if the point is within 1,2, or 3 points after a double fault ###

In [36]:
# def after_doublefault(rows):
#     is_double=rows.isDouble==True
#     results=rows.loc[is_double, 'PtWinner']

df_trim["after_double"] = df_trim["isDouble"]
df_trim["after_double1"] = df_trim["after_double"].shift(periods=1, fill_value=False)
df_trim["after_double2"] = df_trim["after_double1"].shift(periods=1, fill_value=False)
df_trim["after_double3"] = df_trim["after_double2"].shift(periods=1, fill_value=False)

/Users/jamesmoss/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/jamesmoss/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jamesmoss/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

<IPython.core.display.Javascript object>

In [37]:
df_trim.head(3)

,match_id,player1_id,player2_id,year,Svr,isDouble,isAce,PtWinner,isSvrWinner,after_double,after_double1,after_double2,after_double3
0,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,1,1,False,False,False,False
1,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,2,0,False,False,False,False
2,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,2,0,False,False,False,False


<IPython.core.display.Javascript object>

In [38]:
# df_trim.loc[
#     df_trim["after_double"] == True, ["after_double1", "after_double2", "after_double3"]
# ] = True

<IPython.core.display.Javascript object>

In [39]:
df_trim.loc[df_trim["after_double1"] == True, ["after_double2", "after_double3"]] = True

/Users/jamesmoss/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<IPython.core.display.Javascript object>

In [40]:
df_trim.loc[df_trim["after_double2"] == True, ["after_double3"]] = True

<IPython.core.display.Javascript object>

In [41]:
doubles = df_trim[df_trim["after_double3"] == True]
doubles.head()

,match_id,player1_id,player2_id,year,Svr,isDouble,isAce,PtWinner,isSvrWinner,after_double,after_double1,after_double2,after_double3
60,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,True,1,1,False,True,True,True
61,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,1,1,False,False,True,True
62,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,False,False,1,1,False,False,False,True
64,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,2,False,False,2,1,False,True,True,True
65,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,2,False,False,2,1,False,False,True,True


<IPython.core.display.Javascript object>

In [42]:
df_trim[df_trim["isDouble"] == True].head()

,match_id,player1_id,player2_id,year,Svr,isDouble,isAce,PtWinner,isSvrWinner,after_double,after_double1,after_double2,after_double3
59,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,True,False,2,0,True,False,False,False
63,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,2,True,False,1,0,True,False,False,False
85,20200307-M-Davis_Cup_WG_II-RR-Ernests_Gulbis-M...,Ernests_Gulbis,Mohamed_Safwat,2020,1,True,False,2,0,True,False,False,False
128,20200307-M-Davis_Cup_Qualifiers-RR-Alexander_B...,Alexander_Bublik,Robin_Haase,2020,1,True,False,2,0,True,False,False,False
159,20200307-M-Davis_Cup_Qualifiers-RR-Alexander_B...,Alexander_Bublik,Robin_Haase,2020,1,True,False,2,0,True,False,False,False


<IPython.core.display.Javascript object>

TODO: maybe I should separate the after doubles by player 1 and 2?